In [1]:
from matplotlib.pyplot import *
import numpy as np
import h5py
from tqdm import tqdm
#from src.algorithms import *
#import torch
import sklearn
%load_ext autoreload

In [2]:
find_index = lambda wavelenghts, w : np.argmin(np.abs(wavelenghts - w))

In [3]:
def Raman_data_loader(filename):

    with h5py.File(filename, 'a') as output_file:
         
        properties = output_file['properties']
        
        exp_properties = {'step_size' : np.array(properties['step_size'])[0],
                          'speed' : np.array(properties['speed']),
                          'n_points' : np.array(properties['n_points'])
            }
        
        wavelengths = np.array(output_file['properties']['x_data'])
        
        
        spot_numbers = [int(s.split('_')[-1]) for s in list(output_file['data'].keys()) if 'spot' in s ]
        
        Nx,Ny = output_file['properties']['n_points'][0], output_file['properties']['n_points'][1]
        Nl = len(wavelengths)
        spectral_signal = np.zeros([Nx,Ny,Nl])
        
        
        for _i, spot_number in enumerate(spot_numbers):

            ix, iy = int(spot_number//Ny), int(spot_number%Ny)
            
            spot = 'spot_'+str(spot_number)
            data = np.array(output_file['data'][spot]['raw_data'])
                      
            spectral_signal[ix,iy,:] = data
            

    return spectral_signal, wavelengths, exp_properties

In [4]:
filename = "2023913_1110.h5"
#filename = "2023811_1451.h5"
spectrum_raman, wavelengths_raman, exp_properties = Raman_data_loader(filename)

In [5]:
from scipy import sparse
from scipy.sparse.linalg import spsolve

def baseline_als_optimized(y, lam, p, niter = 10):
    L = len(y)
    D = sparse.diags([1, -2, 1],[0, -1, -2], shape = (L, L - 2))
    D = lam * D.dot(D.transpose()) # Precompute this term since it does not depend on `w`
    w = np.ones(L)
    W = sparse.spdiags(w, 0, L, L)
    for i in range(niter):
        W.setdiag(w) # Do not create a new matrix, just update diagonal values
        Z = W + D
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1 - p) * (y < z)
    return z

In [ ]:
lam = 1e2
p = 1e-1

pro_data = 1*spectrum_raman
for i in tqdm(range(0, pro_data.shape[0])):
    for j in range(0, pro_data.shape[1]):
        # print(i,j,end='\r')
        spec = 1 * pro_data[i, j, :]
        pro_data[i, j, :] = spec - baseline_als_optimized(spec, lam = lam, p = p, niter = 10)

 42%|█████████████████████████████████▌                                              | 105/250 [04:48<05:54,  2.44s/it]

In [ ]:
pro_data_norm=pro_data.copy()
for i in range(0,pro_data.shape[0]):
    for j in range(0,pro_data.shape[1]):
        spec = 1 * pro_data[i,j, :]
        sum_spec=np.sum(spec)
        pro_data_norm[i,j,:]=spec/sum_spec

In [ ]:
offset_l = 31
offset_m = 328

mask = pro_data_norm[:, :, offset_l:offset_m].reshape(pro_data.shape[0]*pro_data.shape[1], -1)

# threshold = 120
# threshold2 = 0.01

# # Scaled to max value for each wavenumber
# # masks = np.array([mask[:, i]*(mask[:, i] > threshold)/np.max(mask[:, i]) for i in range(mask.shape[-1])])
# masks = np.array([np.array((mask[:, i]/np.max(mask[:, i]))*((mask[:, i] > threshold)) > threshold2, dtype = int) for i in range(mask.shape[-1])])

In [ ]:
threshold = 1.27e-9

mask_offset = np.array(mask)
mask_t = np.array([(mask_offset[:, i])*((mask_offset[:, i] > threshold)) for i in range(mask_offset.shape[-1])])

In [ ]:
init_k=[0,1]
mask_min_max = np.array([(mask_t[i, :] - mask_t[i, :].min(axis = 0)+ 1e-5)/(mask_t[i, :].max(axis = 0) - mask_t[i, :].min(axis = 0) + 1e-5) for i in range(mask_t.shape[0])])
# mask_min_max = np.array([(mask_t[i, :])/(mask_t[i, :].max(axis = 0) + 1e-10) for i in range(mask_t.shape[0])])

In [ ]:
def read_ref(mineral):
    df = pd.read_csv(mineral + '_raman.txt',header=13,names=['wavenumber','int'],skipfooter=4,engine='python' )
    return np.array(df['wavenumber']),np.array(df['int'])

## Cálculo da entropia

In [ ]:
print(mask_min_max.shape)
entropy_masks=np.array([mask_min_max[i,:]*255 for i in range(297)])

In [ ]:
entropy_masks[10]

In [ ]:
def entropy_calc(map_gr):
    import cv2
    from scipy.stats import entropy

    #image

    _bins = 200

    hist, _ = np.histogram(map_gr, bins=_bins, range=(0, _bins))

    prob_dist = hist / hist.sum()
    image_entropy = entropy(prob_dist, base=2)
    return image_entropy

In [ ]:
offset_l = 31
offset_m = 328
wavelengths_raman.shape
wavelengths=wavelengths_raman[offset_l:offset_m]

minerals = {
    
    'Albite':[508.1],
    'Background':[733.1],
    'Petalite':[491.3],
    'Quartz':[463.92],
    'Spodumene':[704.5]}

mineral_list = list(minerals.keys())

find_index = lambda wavelenghts,w : np.argmin(np.abs(wavelengths-w))

significant_w=[]
for i,mineral in enumerate(mineral_list):
    wn=find_index(wavelengths,minerals[mineral][0])
    significant_w.append(wn)

In [ ]:
image_entropy=[]
for i in range(297):
    map_gr=entropy_masks[i]
    image_entropy.append(entropy_calc(map_gr))


In [ ]:
import matplotlib.pyplot as plt
plt.hist(image_entropy,bins=20)
colors=['red','pink','black','green','brown']
for i in range(len(significant_w)):
    plt.vlines(image_entropy[significant_w[i]],0,40,color=colors[i],label=mineral_list[i])
    
plt.legend()

## Entropias intermédias é a que mantém mais informação - ajustar a gama

In [ ]:
plt.hist(image_entropy,bins=20)
colors=['red','pink','black','green','brown']
for i in range(len(significant_w)):
    plt.vlines(image_entropy[significant_w[i]],0,40,color=colors[i],label=mineral_list[i])
    
plt.xticks([3,4,5,5.2,5.4,5.6,5.8,6])
    
plt.legend()

In [ ]:
relevant_entropies=[]
for i in significant_w:
    relevant_entropies.append(image_entropy[i])
    
#max_en=np.max(relevant_entropies)
#min_en=np.min(relevant_entropies)
min_en=5.0
max_en=6.0

new_maps_cut=[]
for i in range(entropy_masks.shape[0]):
    if min_en<=image_entropy[i]<=max_en:
        new_maps_cut.append(entropy_masks[i,:])
        
new_maps_cut=np.array(new_maps_cut)


In [ ]:
from sklearn import cluster
kmeans = sklearn.cluster.KMeans(n_clusters = 5)
a=kmeans.fit_predict(new_maps_cut)
labels = kmeans.labels_
fig, ax = subplots(2, 3, figsize = (10, 8))
# clusters_c = np.array(kmeans.cluster_centers_.cpu())
clusters_c = np.array(kmeans.cluster_centers_)

j = 0
l = 0

for i in range(len(np.unique(labels))):
    axs = ax[l, j]
    # axs.imshow(clusters_c[i].reshape(pro_data.shape[:2]).T, interpolation = 'none')
    axs.imshow(clusters_c[i].reshape(pro_data.shape[0], pro_data.shape[1]).T, interpolation = 'none')
    axs.set_ylabel('y (mm)')
    axs.set_xlabel('x (mm)')
    axs.grid(False)
    j += 1
    if j == 3:
        j = 0
        l += 1

fig.tight_layout()

In [ ]:
maps=new_maps_cut

In [ ]:
print(maps.shape)
print(maps[100])

## Kmeans with cross entropy

## Entropy equalization

In [ ]:
#P=maps.mean(axis=0)
#P=np.random.random(25000)*255
P=maps[significant_w[3]]
P.sort()
final_masks=[]
for i in range(maps.shape[0]):
    X=maps[i].copy()
    #X.sort()
    sorted_args=X.argsort()
    new_X=np.ones(maps.shape[1])
    for j in range(maps.shape[1]):
        #new_X[j]=float(P[sorted_args[j]])
        new_X[sorted_args[j]]=P[j]
        
    #print(new_X)
    final_masks.append(new_X)
final_masks=np.array(final_masks)

In [ ]:
print(entropy_calc(final_masks[12,:]))

In [ ]:
imshow(final_masks[13].reshape(250,100))


### classificação

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters = 5)
a=kmeans.fit_predict(final_masks)
labels = kmeans.labels_
fig, ax = subplots(2, 3, figsize = (10, 8))
# clusters_c = np.array(kmeans.cluster_centers_.cpu())
clusters_c = np.array(kmeans.cluster_centers_)

j = 0
l = 0

for i in range(len(np.unique(labels))):
    axs = ax[l, j]
    # axs.imshow(clusters_c[i].reshape(pro_data.shape[:2]).T, interpolation = 'none')
    axs.imshow(clusters_c[i].reshape(pro_data.shape[0], pro_data.shape[1]).T, interpolation = 'none')
    axs.set_ylabel('y (mm)')
    axs.set_xlabel('x (mm)')
    axs.grid(False)
    j += 1
    if j == 3:
        j = 0
        l += 1

fig.tight_layout()

## cortar o background

In [ ]:
maps=new_maps_cut

In [ ]:
reshaped_maps=maps.reshape(-1,250,100)

In [ ]:
reshaped_maps.shape

In [ ]:
imshow(reshaped_maps[0])

In [ ]:
fig,ax=subplots(1,2)
ax[0].imshow(reshaped_maps[13])
#cortes
maps_without_background=reshaped_maps[13,5:220,7:75]
ax[1].imshow(maps_without_background)

In [ ]:
maps_without_background_1=reshaped_maps[:,5:220,7:75]
maps_without_background=maps_without_background_1.reshape(maps_without_background_1.shape[0],maps_without_background_1.shape[1]*maps_without_background_1.shape[2])


## classificação

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters = 5)
a=kmeans.fit_predict(maps_without_background)
labels = kmeans.labels_
fig, ax = subplots(2, 3,figsize=(10,8))
# clusters_c = np.array(kmeans.cluster_centers_.cpu())
clusters_c = np.array(kmeans.cluster_centers_)

j = 0
l = 0

for i in range(len(np.unique(labels))):
    axs = ax[l, j]
    # axs.imshow(clusters_c[i].reshape(pro_data.shape[:2]).T, interpolation = 'none')
    axs.imshow(clusters_c[i].reshape(maps_without_background_1.shape[1], maps_without_background_1.shape[2]).T, interpolation = 'none')
    axs.set_ylabel('y (mm)')
    axs.set_xlabel('x (mm)')
    axs.grid(False)
    j += 1
    if j == 3:
        j = 0
        l += 1

fig.tight_layout()

# OLD

## classificação a usar todos os mapas com entropias equalizadas

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters = 6)
a=kmeans.fit_predict(final_masks)
labels = kmeans.labels_
fig, ax = subplots(2, 3, figsize = (10, 8))
# clusters_c = np.array(kmeans.cluster_centers_.cpu())
clusters_c = np.array(kmeans.cluster_centers_)

j = 0
l = 0

for i in range(len(np.unique(labels))):
    axs = ax[l, j]
    # axs.imshow(clusters_c[i].reshape(pro_data.shape[:2]).T, interpolation = 'none')
    axs.imshow(clusters_c[i].reshape(pro_data.shape[0], pro_data.shape[1]).T, interpolation = 'none')
    axs.set_ylabel('y (mm)')
    axs.set_xlabel('x (mm)')
    axs.grid(False)
    j += 1
    if j == 3:
        j = 0
        l += 1

fig.tight_layout()

## cortar primeiro os mapas com entropias fora das zonas e depois fazer a equalização

In [ ]:
relevant_entropies=[]
for i in significant_w:
    relevant_entropies.append(image_entropy[i])
    
max_en=np.max(relevant_entropies)
min_en=np.min(relevant_entropies)

new_maps_cut=[]
for i in range(entropy_masks.shape[0]):
    if min_en<=image_entropy[i]<=max_en:
        new_maps_cut.append(entropy_masks[i,:])
        
new_maps_cut=np.array(new_maps_cut)


In [ ]:
#P=new_maps_cut.mean(axis=0)
P=np.random.random(25000)
#sorted_args=P.argsort()
P.sort()
final_masks=[]
for i in range(new_maps_cut.shape[0]):
    X=new_maps_cut[i].copy()
    #X.sort()
    sorted_args=X.argsort()
    new_X=np.ones(new_maps_cut.shape[1])
    for j in range(new_maps_cut.shape[1]):
        #new_X[j]=float(P[sorted_args[j]])
        new_X[sorted_args[j]]=P[j]
        
    #print(new_X)
    final_masks.append(new_X)
    
final_masks=np.array(final_masks)*255

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters = 4)
a=kmeans.fit_predict(final_masks)
labels = kmeans.labels_
fig, ax = subplots(2, 2, figsize = (10, 8))
# clusters_c = np.array(kmeans.cluster_centers_.cpu())
clusters_c = np.array(kmeans.cluster_centers_)

j = 0
l = 0

for i in range(len(np.unique(labels))):
    axs = ax[l, j]
    # axs.imshow(clusters_c[i].reshape(pro_data.shape[:2]).T, interpolation = 'none')
    axs.imshow(clusters_c[i].reshape(pro_data.shape[0], pro_data.shape[1]).T, interpolation = 'none')
    axs.set_ylabel('y (mm)')
    axs.set_xlabel('x (mm)')
    axs.grid(False)
    j += 1
    if j == 2:
        j = 0
        l += 1

fig.tight_layout()